In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Load the MNIST dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Display the type of the mnist object
print(type(mnist))
mnist.train.num_examples
mnist.train.num_examples

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>


55000

In [2]:
# This function returns a tf.Variable used to store weights in a filter
# The values are initialized with values that can be used to randomize weights

def init_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [3]:
# This function returns a tf.Variable used to store bias terms
# The variable is initialized with values that can be used to initialize biases

def init_bias(bias_shape):
    init_vals = tf.constant(0.1, shape=bias_shape)
    return tf.Variable(init_vals)

In [4]:
# Set Up Convolution Layer and Perform Convolution Computation: Dot Product (X * W)
def create_convolution_layer_and_compute_dot_products(inputs, filter_shape):
    # Initialize the weights = initialize filters
    filters = tf.Variable(tf.random.normal(filter_shape))

    # Create a convolution layer
    conv_layer_outputs = tf.nn.conv2d(inputs, filters, strides=[1, 1, 1, 1], padding='SAME')

    return conv_layer_outputs

In [5]:
# setup a Relu layer and perform computation: Dot product + Bias(x.W+b)
def create_convolution_layer_and_compute_dotproducts_plus_b(inputs, filter_shape):
    # Initialize bias for each output channel
    b= init_bias([filter_shape[3]])
    
    # Perform the computation first by adding: inputs (x * W) + b
    relu_layer_outputs = tf.nn.relu(inputs + b)  
           
    return relu_layer_outputs

In [6]:
# Set Up a Pooling Layer and Reduce Spatial Size

# 1) Define Function to Create Pooling Layer
# 2) Then, to reduce the spatial size of the inputs

# Pooling method: Max pooling
# Kernel size: 2 x 2
# Stride: 2

def create_maxpool2by2_and_reduce_Spatial_size(inputs):
    # Parameters:
    # inputs: Outputs of the previous convolutional layer
    
    # Create maxpool2by2 and reduce_size
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    return pooling_layer_outputs

In [7]:
# Define a function to compute the fully connected layer
def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs, output_size):
  input_size = int(inputs.get_shape()[1])
  W=init_weights([input_size, output_size])
  b=init_bias([output_size])

  fc_xW_plus_bias_outputs=tf.matmul(inputs,W)+b

  return fc_xW_plus_bias_outputs

In [8]:
#Create placeholders for Input and Labels: x and y_true
x=tf.placeholder(tf.float32,shape=[None,784])
y_true=tf.placeholder(tf.float32,shape=[None,10])

In [9]:
#Reshape the input layer
x_image=tf.reshape(x,[-1,28,28,1])

In [10]:
# Create the first convolutional layer, ReLu layer, and perform Computation: x*W+b
conv_layer_1_output= create_convolution_layer_and_compute_dot_products(x_image, filter_shape=[5, 5, 1, 32])

conv_relu_layer_1_outputs=create_convolution_layer_and_compute_dotproducts_plus_b(conv_layer_1_output, filter_shape=[5,5,1,32])

In [11]:
# Create first pooling layer and reduce size
pooling_layer_1_outputs=create_maxpool2by2_and_reduce_Spatial_size(conv_relu_layer_1_outputs)

In [12]:
# Reshape/flatten data and make it ready to be fed in first FC layer
pooling_layer_1_outputs_flat=tf.reshape(pooling_layer_1_outputs,[-1,14*14*32])

In [13]:
# Create first FC Layer, ReLu layer, and output data to dropout layer
fc_layer_1_outputs=create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_1_outputs_flat, output_size=1024)

fc_relu_layer_1_outputs=tf.nn.relu(fc_layer_1_outputs)

In [14]:
# Create a dropout layer and Dropout a fraction of outputs randomly
hold_prob=tf.placeholder(tf.float32)
fc_dropout_outputs=tf.nn.dropout(fc_relu_layer_1_outputs,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
# Create final FC layer 
y_pred=create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

In [16]:
# Create Loss Function and Calculate Softmax cross entropy Loss
softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)

# Compute the mean of losses
cross_entropy_mean=tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [17]:
# Create an optimizer to Optimize CNN Model and Set Learning Rate
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

In [18]:
# Create a trainer to trainig CNN Model
cnn_trainer=optimizer.minimize(cross_entropy_mean)

In [19]:
# Train and test CNN deep learning Model on MNIST dataset

# Create a varaible to initializer to initialize all variables
vars_initializer=tf.global_variables_initializer()

#Set the steps
steps=3000

# Run tf.session() to train and test deep learning CNN model
with tf.Session() as sess:
    sess.run(vars_initializer)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)

        # Train the model
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})

        # Test the model at each 100 step
        if i % 100 == 0:
          print('ON STEP: {}'.format(i))
          print('ACCURACY: ')

          # Compare to find the matches of y_pred and y_true
          matches=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

          # Calculate the accuracy
          acc=tf.reduce_mean(tf.cast(matches,tf.float32))

          # Test the model at each 100th step
          # Using the test data
          # Dropout: NONE because of test, not training
          test_accuracy=sess.run(acc, feed_dict={x: mnist.test.images, y_true: mnist.test.labels, hold_prob: 1.0})

          print(test_accuracy)
          print('\n')



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

ON STEP: 0
ACCURACY: 
0.1963


ON STEP: 100
ACCURACY: 
0.9124


ON STEP: 200
ACCURACY: 
0.9251


ON STEP: 300
ACCURACY: 
0.9282


ON STEP: 400
ACCURACY: 
0.94


ON STEP: 500
ACCURACY: 
0.9487


ON STEP: 600
ACCURACY: 
0.9428


ON STEP: 700
ACCURACY: 
0.9481


ON STEP: 800
ACCURACY: 
0.9483


ON STEP: 900
ACCURACY: 
0.9576


ON STEP: 1000
ACCURACY: 
0.9537


ON STEP: 1100
ACCURACY: 
0.9586


ON STEP: 1200
ACCURACY: 
0.9635


ON STEP: 1300
ACCURACY: 
0.9619


ON STEP: 1400
ACCURACY: 
0.9606


ON STEP: 1500
ACCURACY: 
0.9637


ON STEP: 1600
ACCURACY: 
0.9605


ON STEP: 1700
ACCURACY: 
0.961


ON STEP: 1800
ACCURACY: 
0.9605


ON STEP: 1900
ACCURACY: 
0.9633


ON STEP: 2000
ACCURACY: 
0.9627


ON STEP: 2100
ACCURACY: 
0.9665


ON STEP: 2200
ACCURACY: 
0.969


ON STEP: 2300
ACCURACY: 
0.9714


ON STEP: 2400
ACCURACY: 
0.9697


ON STEP: 2500
ACCURACY: 
0.9739


ON STEP: 2600
ACCURACY: 
0.9732


ON STEP: 2700
ACCURACY: 
0.9698


ON STEP: 2800
ACCURACY: 
0.9725


ON STEP: 2900
ACCURACY: 
0.972